In [1]:
%cd ../
%matplotlib inline

/home/zulqarnain/Code/ntfa_methods_rewrited_batching/htfatorch


In [2]:
import collections
import logging
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils
import glob
import os
import webdataset as wds
import torch

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [4]:
affvids_dir = '/data/zulqarnain/fmri_data/AVFP/'

task_log_csvs = 'AVFP_logfiles'

nifti_files = 'AVFP_rose/'

TASK_ONSET_DELAY = 3

AFFVID_FILENAME_TEMPLATE = 'sub-%03d_run-%01d_AVFP_denoised.nii.gz'

AFFVIDS_FILE = 'data/avfp_memory_mini.tar'

tar_file = AFFVIDS_FILE
sink = wds.TarWriter(tar_file)


def affvid_filename(subject, run):
    return affvids_dir + nifti_files + (AFFVID_FILENAME_TEMPLATE % (subject, run))

class TaskElement:
    def __init__(self, task, start, end, run, fear_rating=None):
        def round_off_time(t):
            if t is not None:
                if task != 'rest':
                    return round((t + TASK_ONSET_DELAY) / 0.8)
                else:
                    return round(t)
            else:
                return None
        self.task = task
        self.start_time = round_off_time(start)
        self.end_time = round_off_time(end)
        self.run = run
        self.fear_rating = fear_rating

def parse_task_lines(lines):
    for (i, line) in enumerate(lines):
        cols = line.split(' ')
        task = cols[0]
        start_time = float(cols[5])
        end_time = float(cols[6])
        run = int(cols[4])
        fear_rating = float(cols[10])
        yield TaskElement(task, start_time, end_time, run, fear_rating)

def rest_tasks(tasks):
    yield TaskElement('rest', 0, tasks[0].start_time - 1, tasks[0].run)
    for i in range(1, len(tasks)):
        rest_start = tasks[i-1].end_time + 1
        rest_end = tasks[i].start_time - 1
        if tasks[i].run == tasks[i-1].run:
            yield TaskElement('rest', rest_start, rest_end, tasks[i].run)
        else:
            yield TaskElement('rest', rest_start, None, tasks[i-1].run)
            yield TaskElement('rest', 0, rest_end, tasks[i].run)
    yield TaskElement('rest', tasks[-1].end_time + 1, None, tasks[-1].run)

bad_runs = collections.defaultdict(lambda: [])
# bad_runs[3] = [1, 2, 3]
# bad_runs[7] = [3]
# bad_runs[14] = [3]
# bad_runs[22] = [1, 2, 3]
# bad_runs[23] = [1]
# bad_runs[24] = [1, 2, 3]
# bad_runs[26] = [3]

## exclude these to create a mini

exclude_subjects = [119, 107, 103, 149, 145, 137, 133, 125, 121, 163, 151, 108, 146, 134,
                    122, 164, 160, 114, 110, 140, 139, 111, 153, 128, 124, 154, 112, 116,
                    162, 120, 170, 132, 136, 144, 106, 118, 143, 144, 106, 118, 143, 147,
                    100, 104, 105, 115, 117, 123, 127, 129, 130, 135, 142]
exclude_tasks = ['rest', 'Spiders_11.m4v']

def read_tasks(task_csv):
    def sentinel(f):
        return f if f is not None else 0.0
    with open(task_csv, 'r') as task_csv_file:
        subject = int(task_csv.split('_')[-1].strip('.txt'))
        logging.info('Subject %d', subject)
        task_lines = list(parse_task_lines(task_csv_file.readlines()))
        task_lines += list(rest_tasks(task_lines))
        rest_lines = [r for r in task_lines if r.task == 'rest']
        rest_lines = sorted(rest_lines, key=lambda t: sentinel(t.run))
        rest_starts_dict = {key: [] for key in range(1, 3)}
        rest_ends_dict = {key: [] for key in range(1, 3)}
        for (i,rest) in enumerate(rest_lines):
            if (rest.run in bad_runs[subject]) or (int(subject) in exclude_subjects):
                continue
            if rest.end_time is not None and rest.start_time is not None:
                rest_ends_dict[rest.run].append(rest.end_time)
                rest_starts_dict[rest.run].append(rest.start_time)
        task_lines = sorted(task_lines, key=lambda t: sentinel(t.start_time))
        for (i, task) in enumerate(task_lines):
            if task.run in bad_runs[subject] or (int(subject) in exclude_subjects) or (task.task in exclude_tasks):
                logging.info('Excluding block %d %s of run %d for motion', i,
                             task.task, task.run)
                continue
            logging.info('Block %d %s of run %d started at %f, ended at %f', i,
                         task.task, task.run, sentinel(task.start_time), sentinel(task.end_time))
            result = niidb.FMriActivationBlock(zscore=True, zscore_by_rest=True)
            result.subject = subject
            result.task = task.task
            result.run = task.run
            result.block = i
            result.start_time = task.start_time
            result.end_time = task.end_time
            result.rest_start_times = rest_starts_dict[result.run]
            result.rest_end_times = rest_ends_dict[result.run]
            result.individual_differences = {'fear_rating': task.fear_rating}
            yield result

In [5]:
OVERRIDE = True

if not os.path.exists(AFFVIDS_FILE) or OVERRIDE:
    total_trs = 0
    metadata = {
        'blocks': []
    }
    block_id = 0
    for task_csv in utils.sorted_glob(affvids_dir + task_log_csvs + '/*.txt'):
        for block in read_tasks(task_csv):
            block.filename = affvid_filename(block.subject, block.run)
#             block.mask = mask_filename(block.subject, block.run)
            block.rest_end_times = '[' + ', '.join(map(str, block.rest_end_times)) + ']'
            block.rest_start_times = '[' + ', '.join(map(str, block.rest_start_times)) + ']'
            block.block = block_id
            block_id += 1
            block.mask = '/data/zulqarnain/fmri_data/AVFP/group_mask_N55.nii.gz'
            block.smooth = 6
            block.load()
            metadata['blocks'].append(block.wds_metadata())

            for vals in block.format_wds():
                sink.write(vals)
            block_trs = (block.end_time - block.start_time)
            total_trs += block_trs
    
    metadata['voxel_locations'] = block.locations
    metadata['num_times'] = total_trs
    torch.save(metadata, tar_file + '.meta')
    logging.info('Recorded metadata, including voxel locations')

sink.close()

10/14/2021 10:48:47 Subject 100
10/14/2021 10:48:47 Excluding block 0 rest of run 1 for motion
10/14/2021 10:48:47 Excluding block 1 rest of run 2 for motion
10/14/2021 10:48:47 Excluding block 2 Social_a_1.mp4 of run 1 for motion
10/14/2021 10:48:47 Excluding block 3 Spiders_a_3.m4v of run 2 for motion
10/14/2021 10:48:47 Excluding block 4 rest of run 1 for motion
10/14/2021 10:48:47 Excluding block 5 rest of run 2 for motion
10/14/2021 10:48:47 Excluding block 6 Heights_b_3.mov of run 2 for motion
10/14/2021 10:48:47 Excluding block 7 Social_b_1.mp4 of run 1 for motion
10/14/2021 10:48:47 Excluding block 8 rest of run 2 for motion
10/14/2021 10:48:47 Excluding block 9 rest of run 1 for motion
10/14/2021 10:48:47 Excluding block 10 Social_a_3.mp4 of run 2 for motion
10/14/2021 10:48:47 Excluding block 11 Spiders_b_1.m4v of run 1 for motion
10/14/2021 10:48:47 Excluding block 12 rest of run 2 for motion
10/14/2021 10:48:47 Excluding block 13 rest of run 1 for motion
10/14/2021 10:48:47

10/14/2021 10:48:47 Excluding block 18 Social_b_1.mp4 of run 1 for motion
10/14/2021 10:48:47 Excluding block 19 Social_b_3.mp4 of run 2 for motion
10/14/2021 10:48:47 Excluding block 20 rest of run 2 for motion
10/14/2021 10:48:47 Excluding block 21 rest of run 1 for motion
10/14/2021 10:48:47 Excluding block 22 Social_b_4.mov of run 1 for motion
10/14/2021 10:48:47 Excluding block 23 Heights_b_4.mp4 of run 2 for motion
10/14/2021 10:48:47 Excluding block 24 rest of run 1 for motion
10/14/2021 10:48:47 Excluding block 25 rest of run 2 for motion
10/14/2021 10:48:47 Excluding block 26 Heights_a_2.m4v of run 2 for motion
10/14/2021 10:48:47 Excluding block 27 Social_a_4.mp4 of run 1 for motion
10/14/2021 10:48:47 Excluding block 28 rest of run 1 for motion
10/14/2021 10:48:47 Excluding block 29 rest of run 2 for motion
10/14/2021 10:48:47 Excluding block 30 Social_a_3.mp4 of run 1 for motion
10/14/2021 10:48:47 Excluding block 31 Social_a_1.mp4 of run 2 for motion
10/14/2021 10:48:47 Ex

10/14/2021 10:48:47 Excluding block 36 rest of run 1 for motion
10/14/2021 10:48:47 Excluding block 37 rest of run 2 for motion
10/14/2021 10:48:47 Excluding block 38 Heights_b_3.mov of run 1 for motion
10/14/2021 10:48:47 Excluding block 39 Heights_a_1.mp4 of run 2 for motion
10/14/2021 10:48:47 Excluding block 40 rest of run 1 for motion
10/14/2021 10:48:47 Excluding block 41 rest of run 2 for motion
10/14/2021 10:48:47 Excluding block 42 Social_b_2.mov of run 1 for motion
10/14/2021 10:48:47 Excluding block 43 Spiders_b_1.m4v of run 2 for motion
10/14/2021 10:48:47 Excluding block 44 rest of run 1 for motion
10/14/2021 10:48:47 Excluding block 45 rest of run 2 for motion
10/14/2021 10:48:47 Excluding block 46 Heights_b_4.mp4 of run 2 for motion
10/14/2021 10:48:47 Excluding block 47 Heights_b_2.m4v of run 1 for motion
10/14/2021 10:48:47 Excluding block 48 rest of run 2 for motion
10/14/2021 10:48:47 Excluding block 49 rest of run 1 for motion
10/14/2021 10:48:47 Subject 107
10/14/2

10/14/2021 10:48:47 Excluding block 4 rest of run 2 for motion
10/14/2021 10:48:47 Excluding block 5 rest of run 1 for motion
10/14/2021 10:48:47 Excluding block 6 Heights_b_3.mov of run 1 for motion
10/14/2021 10:48:47 Excluding block 7 Spiders_b_3.mov of run 2 for motion
10/14/2021 10:48:47 Excluding block 8 rest of run 2 for motion
10/14/2021 10:48:47 Excluding block 9 rest of run 1 for motion
10/14/2021 10:48:47 Excluding block 10 Social_a_1.mp4 of run 1 for motion
10/14/2021 10:48:47 Excluding block 11 Spiders_b_1.m4v of run 2 for motion
10/14/2021 10:48:47 Excluding block 12 rest of run 1 for motion
10/14/2021 10:48:47 Excluding block 13 rest of run 2 for motion
10/14/2021 10:48:47 Excluding block 14 Spiders_a_3.m4v of run 1 for motion
10/14/2021 10:48:47 Excluding block 15 Heights_a_1.mp4 of run 2 for motion
10/14/2021 10:48:47 Excluding block 16 rest of run 1 for motion
10/14/2021 10:48:47 Excluding block 17 rest of run 2 for motion
10/14/2021 10:48:47 Excluding block 18 Height

10/14/2021 10:48:47 Excluding block 23 Spiders_a_1.m4v of run 1 for motion
10/14/2021 10:48:47 Excluding block 24 rest of run 2 for motion
10/14/2021 10:48:47 Excluding block 25 rest of run 1 for motion
10/14/2021 10:48:47 Excluding block 26 Spiders_b_2.mp4 of run 1 for motion
10/14/2021 10:48:47 Excluding block 27 Social_b_1.mp4 of run 2 for motion
10/14/2021 10:48:47 Excluding block 28 rest of run 1 for motion
10/14/2021 10:48:47 Excluding block 29 rest of run 2 for motion
10/14/2021 10:48:47 Excluding block 30 Heights_a_2.m4v of run 1 for motion
10/14/2021 10:48:47 Excluding block 31 Spiders_a_2.mp4 of run 2 for motion
10/14/2021 10:48:47 Excluding block 32 rest of run 2 for motion
10/14/2021 10:48:47 Excluding block 33 rest of run 1 for motion
10/14/2021 10:48:47 Excluding block 34 Heights_b_4.mp4 of run 1 for motion
10/14/2021 10:48:47 Excluding block 35 Spiders_a_4.mov of run 2 for motion
10/14/2021 10:48:47 Excluding block 36 rest of run 1 for motion
10/14/2021 10:48:47 Excludin

...

10/14/2021 10:49:30 Excluding block 29 rest of run 2 for motion
10/14/2021 10:49:30 Excluding block 30 Heights_a_1.mp4 of run 1 for motion
10/14/2021 10:49:30 Excluding block 31 Spiders_a_4.mov of run 2 for motion
10/14/2021 10:49:30 Excluding block 32 rest of run 1 for motion
10/14/2021 10:49:30 Excluding block 33 rest of run 2 for motion
10/14/2021 10:49:30 Excluding block 34 Heights_b_3.mov of run 2 for motion
10/14/2021 10:49:30 Excluding block 35 Spiders_b_2.mp4 of run 1 for motion
10/14/2021 10:49:30 Excluding block 36 rest of run 2 for motion
10/14/2021 10:49:30 Excluding block 37 rest of run 1 for motion
10/14/2021 10:49:30 Excluding block 38 Spiders_a_2.mp4 of run 2 for motion
10/14/2021 10:49:30 Excluding block 39 Social_b_3.mp4 of run 1 for motion
10/14/2021 10:49:30 Excluding block 40 rest of run 2 for motion
10/14/2021 10:49:30 Excluding block 41 rest of run 1 for motion
10/14/2021 10:49:30 Excluding block 42 Spiders_b_4.m4v of run 1 for motion
10/14/2021 10:49:30 Excludin

10/14/2021 10:51:02 Block 23 Spiders_a_1.m4v of run 2 started at 255.000000, ended at 281.000000
10/14/2021 10:51:03 Excluding block 24 rest of run 2 for motion
10/14/2021 10:51:03 Excluding block 25 rest of run 1 for motion
10/14/2021 10:51:03 Block 26 Spiders_a_3.m4v of run 2 started at 303.000000, ended at 328.000000
10/14/2021 10:51:03 Block 27 Spiders_b_1.m4v of run 1 started at 304.000000, ended at 329.000000
10/14/2021 10:51:03 Excluding block 28 rest of run 2 for motion
10/14/2021 10:51:03 Excluding block 29 rest of run 1 for motion
10/14/2021 10:51:03 Block 30 Heights_b_3.mov of run 1 started at 351.000000, ended at 377.000000
10/14/2021 10:51:03 Block 31 Heights_a_1.mp4 of run 2 started at 351.000000, ended at 378.000000
10/14/2021 10:51:03 Excluding block 32 rest of run 1 for motion
10/14/2021 10:51:03 Excluding block 33 rest of run 2 for motion
10/14/2021 10:51:03 Block 34 Heights_b_4.mp4 of run 1 started at 398.000000, ended at 423.000000
10/14/2021 10:51:03 Block 35 Socia

10/14/2021 10:51:45 Block 27 Spiders_b_2.mp4 of run 2 started at 303.000000, ended at 329.000000
10/14/2021 10:51:45 Excluding block 28 rest of run 1 for motion
10/14/2021 10:51:45 Excluding block 29 rest of run 2 for motion
10/14/2021 10:51:45 Block 30 Spiders_a_1.m4v of run 1 started at 350.000000, ended at 376.000000
10/14/2021 10:51:45 Block 31 Heights_b_2.m4v of run 2 started at 350.000000, ended at 375.000000
10/14/2021 10:51:45 Excluding block 32 rest of run 2 for motion
10/14/2021 10:51:45 Excluding block 33 rest of run 1 for motion
10/14/2021 10:51:45 Block 34 Spiders_a_4.mov of run 1 started at 399.000000, ended at 424.000000
10/14/2021 10:51:46 Block 35 Heights_a_2.m4v of run 2 started at 399.000000, ended at 425.000000
10/14/2021 10:51:46 Excluding block 36 rest of run 1 for motion
10/14/2021 10:51:46 Excluding block 37 rest of run 2 for motion
10/14/2021 10:51:46 Block 38 Spiders_b_3.mov of run 2 started at 445.000000, ended at 471.000000
10/14/2021 10:51:46 Block 39 Heigh

10/14/2021 10:51:46 Excluding block 42 Heights_a_3.mp4 of run 1 for motion
10/14/2021 10:51:46 Excluding block 43 Spiders_a_1.m4v of run 2 for motion
10/14/2021 10:51:46 Excluding block 44 rest of run 1 for motion
10/14/2021 10:51:46 Excluding block 45 rest of run 2 for motion
10/14/2021 10:51:46 Excluding block 46 Social_a_4.mp4 of run 2 for motion
10/14/2021 10:51:46 Excluding block 47 Heights_b_3.mov of run 1 for motion
10/14/2021 10:51:46 Excluding block 48 rest of run 2 for motion
10/14/2021 10:51:46 Excluding block 49 rest of run 1 for motion
10/14/2021 10:51:46 Subject 164
10/14/2021 10:51:46 Excluding block 0 rest of run 1 for motion
10/14/2021 10:51:46 Excluding block 1 rest of run 2 for motion
10/14/2021 10:51:46 Excluding block 2 Heights_a_2.m4v of run 2 for motion
10/14/2021 10:51:46 Excluding block 3 Social_b_4.mov of run 1 for motion
10/14/2021 10:51:46 Excluding block 4 rest of run 2 for motion
10/14/2021 10:51:46 Excluding block 5 rest of run 1 for motion
10/14/2021 10:

10/14/2021 10:52:29 Block 47 Spiders_a_3.m4v of run 1 started at 542.000000, ended at 567.000000
10/14/2021 10:52:29 Excluding block 48 rest of run 2 for motion
10/14/2021 10:52:29 Excluding block 49 rest of run 1 for motion
10/14/2021 10:52:29 Subject 169
10/14/2021 10:52:29 Excluding block 0 rest of run 1 for motion
10/14/2021 10:52:29 Excluding block 1 rest of run 2 for motion
10/14/2021 10:52:29 Block 2 Social_a_3.mp4 of run 2 started at 18.000000, ended at 43.000000
10/14/2021 10:52:29 Loading Nifti image /data/zulqarnain/fmri_data/AVFP/AVFP_rose/sub-169_run-2_AVFP_denoised.nii.gz with mask /data/zulqarnain/fmri_data/AVFP/group_mask_N55.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
10/14/2021 10:52:58 Block 3 Social_a_4.mp4 of run 1 started at 19.000000, ended at 43.000000
10/14/2021 10:52:58 Loading Nifti image /data/zulqarnain/fmri_data/AVFP/AVFP_rose/sub-169_run-1_AVFP_denoised.nii.gz with mask /data/zulqarnain/fmri_data/AVFP/group_mask_N55.nii.gz (zscore=True, smooth=6, 

In [6]:
import htfa_torch.tardb as tardb

In [7]:
db = tardb.FmriTarDataset(AFFVIDS_FILE)

In [8]:
db.mean_block(save=True)
db.normalize_activations(save=True)

([tensor(7.7681),
  tensor(7.2707),
  tensor(7.2707),
  tensor(7.7681),
  tensor(7.7681),
  tensor(7.2707),
  tensor(7.2707),
  tensor(7.7681),
  tensor(7.2707),
  tensor(7.7681),
  tensor(7.2707),
  tensor(7.7681),
  tensor(7.7681),
  tensor(7.2707),
  tensor(7.2707),
  tensor(7.7681),
  tensor(7.2707),
  tensor(7.7681),
  tensor(7.2707),
  tensor(7.7681),
  tensor(7.2707),
  tensor(7.7681),
  tensor(7.2707),
  tensor(7.7681),
  tensor(9.6371),
  tensor(7.0541),
  tensor(9.6371),
  tensor(7.0541),
  tensor(9.6371),
  tensor(7.0541),
  tensor(7.0541),
  tensor(9.6371),
  tensor(9.6371),
  tensor(7.0541),
  tensor(9.6371),
  tensor(7.0541),
  tensor(9.6371),
  tensor(7.0541),
  tensor(7.0541),
  tensor(9.6371),
  tensor(9.6371),
  tensor(7.0541),
  tensor(7.0541),
  tensor(9.6371),
  tensor(9.6371),
  tensor(7.0541),
  tensor(7.0541),
  tensor(9.6371),
  tensor(13.4272),
  tensor(10.4737),
  tensor(13.4272),
  tensor(10.4737),
  tensor(10.4737),
  tensor(13.4272),
  tensor(13.4272),
  t

In [9]:
logging.info('Finished building TarDb out of AffVids dataset in %s', affvids_dir)

10/14/2021 10:55:52 Finished building TarDb out of AffVids dataset in /data/zulqarnain/fmri_data/AVFP/
